Параметры

In [54]:

endpoint = "http://192.168.116.131:8181/blazegraph/namespace/kb/sparql"
profileNames = ["Example_Вывод из эксплуатации участка линии"]
outputFolder = "C:\Bin\MessageTemplates"

In [55]:
from  utils import *
import json

SPARQL Запросы

In [56]:
def classesQuery(profile):
   return """
   PREFIX sch:  <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX syn:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX owl:  <http://www.w3.org/2002/07/owl#>
   SELECT ?profile ?class ?parent WHERE{
      ?profile sch:label "%s".
      ?profile syn:type owl:Ontology.
      ?class sch:isDefinedBy ?profile.
      ?class syn:type 	owl:Class.
      OPTIONAL{?class sch:subClassOf ?par}
      
      BIND(coalesce(?par,"")  AS ?parent ) .
   }
   """ % profile

In [57]:
def fieldsQuery(profile):
   return """
   PREFIX sch:  <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX syn:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX owl:  <http://www.w3.org/2002/07/owl#>
   PREFIX ext:  <http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#>
   SELECT * WHERE{
      ?profile sch:label "%s".
      ?profile syn:type owl:Ontology.
      ?class sch:isDefinedBy ?profile.
      ?class syn:type owl:Class.
      ?field sch:domain ?class.
      ?field syn:type ?type.
      OPTIONAL{?field sch:range ?range.}
      OPTIONAL{?field ext:multiplicity ?multiplicity.}
   }
   """% profile


Выполнение запросов

In [58]:
classesDF = {}
fieldsDF = {}
classes = {}
fields = {}
for prof in profileNames:
    classesDF[prof] = sparqlDF(endpoint, classesQuery(prof), True)
    fieldsDF[prof] = sparqlDF(endpoint, fieldsQuery(prof), True)
    classes[prof] = dataFrameToObjects(classesDF[prof] )
    fields[prof] = dataFrameToObjects( fieldsDF[prof] )



Результаты запросов

In [59]:
classesDF[profileNames[0]].head(1000)

,profile,class,parent
0,3,IdentifiedObject,
1,3,PowerSystemResource,IdentifiedObject
2,3,ConnectivityNodeContainer,PowerSystemResource
3,3,EquipmentContainer,ConnectivityNodeContainer
4,3,TechPlace,
5,3,Substation,EquipmentContainer
6,3,Line,EquipmentContainer


In [60]:
fieldsDF[profileNames[0]].head(1000)

,profile,class,field,type,range,multiplicity
0,3,IdentifiedObject,Names,ObjectProperty,Name,n
1,3,IdentifiedObject,OrganisationRoles,ObjectProperty,OrganisationRole,n
2,3,IdentifiedObject,aliasName,DatatypeProperty,string,1
3,3,IdentifiedObject,description,DatatypeProperty,string,1
4,3,IdentifiedObject,mRID,DatatypeProperty,string,1
5,3,IdentifiedObject,name,DatatypeProperty,string,1
6,3,PowerSystemResource,AssetDatasheet,ObjectProperty,AssetInfo,1
7,3,PowerSystemResource,Assets,ObjectProperty,Asset,n
8,3,PowerSystemResource,Location,ObjectProperty,Location,1
9,3,PowerSystemResource,Measurements,ObjectProperty,Measurement,n


Построение структуры

In [61]:
mainClasses = {}
for prof in profileNames:
    profClasses = classes[prof]
    profFields = fields[prof]
    concreteClases = list(filter(lambda c1: not any(
        c2["parent"] == c1["class"] for c2 in profClasses), profClasses))
    profMainClasses = list(filter(lambda c: not any(
        f["range"] == c["class"] and f["multiplicity"][-1] == "n" for f in profFields), concreteClases))

    def collectClassFields(leaf, cur):
        for field in filter(lambda f: f["class"] == cur["class"], profFields):
            field["refClass"] = None
            if (field["type"] == "ObjectProperty" and field["multiplicity"][-1] == "n"):
                field["refClass"] = next(
                    (c for c in concreteClases if c["class"] == field["range"]), None)
            leaf["fields"].append(field)

        if (cur["parent"]):
            nxt = next(
                (c for c in profClasses if c["class"] == cur["parent"]), None)
            collectClassFields(leaf, nxt)

    for c in concreteClases:
        c["fields"] = []
        collectClassFields(c, c)

  
    # if (len(profMainClasses) > 1):
    #     raise ValueError(
    #         'Не удалось определить основоной класс в профиле '+prof)
    mainClasses[prof]=profMainClasses
  


Формирование JSON

In [62]:
import os


def addLevel(parent, cls, fieldName=None):
    jobj = {}
    pref = "astu:"
    if not fieldName:
        parent["m:Payload"] = jobj
    else:
        parent[fieldName] = [jobj]
    jobj["@type"] = pref+cls["class"]
    jobj["@id"] = ""

    for f in cls["fields"]:
        fname = pref + cls["class"]+"." + f["field"]
        if (f["refClass"]):
            addLevel(jobj, f["refClass"], fname)
        else:
            jobj[fname] = f["range"]





def createProfMessage(mainClass):
    jmain = {}
    addLevel(jmain, mainClass)
    with open(os.path.join(outputFolder, mainClass["profile"]+"_"+mainClass["class"]+".json"), "w") as outfile:
        json.dump(jmain, outfile, indent=6)
    # print(json.dumps(jmain, indent=6))
    


for prof in profileNames:
    for mainClass in mainClasses[prof]:
        createProfMessage(mainClass)
